# Applying AI to Sudoku

In [54]:
from function import *
from utils import *

In [55]:
%reload_ext autoreload
%autoreload 2

## Solving a easy Sudoku

<img src="images/sudoku.png" width="50%"/>

### Encoding the board
Transform sudoku board into string / dict format for further processing

In [3]:
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

In [4]:
display(grid_values_v1(grid))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


### Strategy 1: Elimination
Constraint the possible values of unfilled boxes by peers' values. Peers are divided into three types: 
1. Row unit
2. Column unit
3. 3 x 3 Square unit

<img src="images/sudoku_eliminate.png" width="50%"/>

#### Replace '.' with all possible values

In [5]:
display(grid_values(grid))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

#### After elimination

In [13]:
display(eliminate(grid_values(grid)))

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


### Strategy 2: Only Choice
If a digit only appear once in a box as candidate in a unit, we're assured that the digit should be in the unfilled box because there is no other possible places/boxes to put the digit.

<img src="images/sudoku_only_choice.png" width="50%"/>


In [46]:
display(only_choice(eliminate(grid_values(grid))))

  45   4578   3   |  49    2     1   |  6    5789   57  
  9     6     47  |  3     47    5   |  8    278    1   
  2    257    1   |  8     79    6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17   1467   9   
  6     9     5   |  4     1     47  |  3     8    2467 


### Constraint Propagation

In [59]:
display(reduce_puzzle(grid_values(grid)))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


<img src="images/sudoku_answer.png" width="50%"/>


## Harder Sudoku

<img src="images/harder-puzzle.png" width="50%"/>


In [66]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid2)
display(reduce_puzzle(values))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


### Strategy 3: Search
Use DFS first.

<img src="images/DFS.png" width="50%"/>


In [84]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if not values: return values # fail case
    
    # Choose one of the unfilled squares with the fewest possibilities
    box, digits = min(values.items(),
                      key=lambda x: len(x[1]) if len(x[1]) != 1 else 10)
    
    if len(digits) == 1:
        # print('Sudoku solved.(no square with > 1 digit)')
        return values
    else:
        for digit in digits:
            values_simulate = dict(values)
            values_simulate[box] = digit
            attempt = search(values_simulate)
            if attempt:
                return attempt


In [85]:
display(search(values))

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


## Naked Twins

## Diagonal Sudoku